In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import scipy.stats as stats
from sklearn.metrics import r2_score, accuracy_score, fbeta_score
from sklearn.model_selection import RandomizedSearchCV

df = pd.read_stata(r"D:\World Bank\Honduras PMT benchmark\Data_out\CONSOLIDADA_2023_clean.dta")

In [2]:
# df["EDAD2"] = df["EDAD"]**2
# oh1 = pd.get_dummies(df['CIVIL'], prefix="CIVIL_", dummy_na=True)
# oh2 = pd.get_dummies(df['CH307'], prefix="dis_", dummy_na=True)
# oh3 = pd.get_dummies(df['CH308'], prefix="orig_", dummy_na=True)
# oh4 = pd.get_dummies(df['OC609'], prefix="trab1_", dummy_na=True)
# oh5 = pd.get_dummies(df['CATEGOP'], prefix="trab2_", dummy_na=True)
# oh6 = pd.get_dummies(df['RAMAOP'], prefix="trab3_", dummy_na=True)
# oh7 = pd.get_dummies(df['OCUPAOP'], prefix="trab4_", dummy_na=True)
# oh8 = pd.get_dummies(df['DOMI'], prefix="domi_", dummy_na=True)
# oh9 = pd.get_dummies(df['SEXO'], prefix="genero_", dummy_na=True)
# oh10 = pd.get_dummies(df['ED01'], prefix="ed_", dummy_na=True)
# oh11 = pd.get_dummies(df['CA501'], prefix="ca501_", dummy_na=True)
# oh12 = pd.get_dummies(df['UR'], prefix="ur_", dummy_na=True)
# oh13 = pd.get_dummies(df['NBI'], prefix="nbi_", dummy_na=True)
# one_hots = pd.concat([oh1, oh2, oh3, oh4, oh5, oh6, oh7, oh8, oh9, oh10, oh11, oh12, oh13], axis=1)
# one_hots_cols = one_hots.columns.to_list()
# df = pd.concat([df, one_hots], axis=1)
# df = pd.concat([df], axis=1)

In [3]:
# Variables globales para las distintas estimaciones **
vars_pmtoriginal = ["Ocupacion_bien", "Paredes_bien", "Pension_bien", "Refri_mal", "Remesas_bien", "Aire_mal", "Alquileres_bien", "Alumbrado_bien", "Basura_bien", "Carro_mal", "Cocina2_bien", "Compu_mal", "Dependencia", "dv111", "Ed_diversif_bien", "Ed_univer_bien", "edad_0_5", "edad_15_21", "edad_60_120", "edad_6_14", "Estufa_mal", "Vivienda2_bien", "EqSonido_mal", "HaySanitario_bien", "Sanitario_bien", "Civil_mal", "Cocina_bien", "Cable_mal", "Hacinamiento", "Moto_mal", "Piso_mal", "Bici_mal", "Exterior_bien", "Dominio_1", "Dominio_2", "Dominio_3", "Vivienda_bien", "Agua2_bien", "Agua_bien", "TV_mal", "Telefono_mal"]
IPM_vars = ["privacion_agua_h", "privacion_saneamiento_h", "privacion_cocina_h", "privacion_educ_h", "privacion_asistencia_h", "privacion_alfab_h", "privacion_elec_h", "privacion_piso_h", "privacion_techo_h", "privacion_segsoc_h", "privacion_desocup_h", "privacion_subemp_h", "privacion_ocup_h", "privacion_trabinf_h", "privacion_trabadol1_h", "privacion_trabadol2_h", "privacion_pared_h", "privacion_hacina_h"]
vars_conjunto1 = vars_pmtoriginal + IPM_vars

#* Eliminamos las variables que eran muy multicolineales, eran falseables o no dependian de los hogares
vars_conjunto2 = ["privacion_saneamiento_h","privacion_cocina_h","privacion_educ_h","privacion_asistencia_h","privacion_alfab_h","Piso_mal","Paredes_bien","EqSonido_mal","HaySanitario_bien","Sanitario_bien","Cocina2_bien","Hacinamiento","Cable_mal","Moto_mal","Bici_mal","Dominio_1","Dominio_2","Dominio_3","Pension_bien","Refri_mal","Aire_mal","Carro_mal","Compu_mal","dv111","Ed_diversif_bien","Ed_univer_bien","edad_0_5","edad_15_21","edad_60_120","edad_6_14","Estufa_mal","Agua2_bien","Dependencia"]

In [10]:
def fit_linear_model(df, y_var, vars):
    """ Fit a linear model to the data """
    
    from sklearn.linear_model import LinearRegression
    
    df_test = df[df.test_set==1]
    df_test = df_test[vars + [y_var, "FACTOR_P"]].dropna()
    df_train = df[df.test_set==0]
    df_train = df_train[vars + [y_var, "FACTOR_P"]].dropna()
    X_test = df_test[vars]
    y_test = df_test[y_var]
    w_test = df_test["FACTOR_P"]
    X_train = df_train[vars]
    y_train = df_train[y_var]
    w_train = df_train["FACTOR_P"]
    
    model = LinearRegression()
    model.fit(X_train, y_train, sample_weight=w_train)
    y_test_preds = model.predict(X_test)
    y_train_preds = model.predict(X_train)
        
    return model, y_test_preds, y_train_preds, y_test, y_train, w_test, w_train
    
    

def fit_xgboost_reg(df, y_var, vars, params_grid, scoring="neg_mean_absolute_error"):
    """
    Trains and evaluates an XGBoost regression model using a randomized grid search for hyperparameter tuning.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the dataset with both training and test sets, including features and target variables.
        The DataFrame should have a column 'test_set' where 0 indicates training data and 1 indicates test data.
    
    y_var: str
        A column name representing the variable to use as labels of the model.

    vars : list
        A list of column names representing the independent variables (features) used for training the model.
        
    params_grid : dict
        Dictionary where keys are XGBoost hyperparameters and values are lists of possible values for those
        hyperparameters. This grid is used for randomized hyperparameter tuning.

    Returns:
    --------
    random_search : RandomizedSearchCV object
        The fitted RandomizedSearchCV object containing the best estimator and results from the grid search.
        
    Workflow:
    ---------
    1. Splits the input DataFrame into training and test sets based on the 'test_set' column.
    2. Prepares the feature matrix (X) and target variable (y) for both training and test sets.
    3. Converts the data into DMatrix format required for XGBoost.
    4. Initializes an XGBoost regressor with the 'hist' tree method and 'cuda' for GPU acceleration.
    5. Conducts a RandomizedSearchCV to tune hyperparameters based on a given parameter grid.
    6. Fits the model to the training data and evaluates its performance on both the training and test sets.
    7. Prints R-squared metrics for both training and test sets.
    8. Returns the fitted RandomizedSearchCV object.
    """

    df_test = df[df.test_set==1]
    df_train = df[df.test_set==0]
    X_test = df_test[vars]
    y_test = df_test[y_var]
    w_test = df_test["FACTOR_P"]
    X_train = df_train[vars]
    y_train = df_train[y_var]
    w_train = df_train["FACTOR_P"]
    
    model = xgb.XGBRegressor(
        tree_method="hist",
        device="cuda",
    )

    random_search = RandomizedSearchCV(
        model, 
        param_distributions=params_grid, 
        n_iter=5, 
        cv=5, 
        scoring='neg_mean_absolute_error', 
        verbose=0, 
    )

    random_search.fit(X_train, y_train)
    y_test_preds = random_search.best_estimator_.predict(X_test)
    y_train_preds = random_search.best_estimator_.predict(X_train)
    
    return random_search, y_test_preds, y_train_preds, y_test, y_train, w_test, w_train

def asigna_beneficios(y, weights, percentage):
    df = pd.DataFrame({"y": y, "weights": weights})
    df = df.sort_values("y")

    ranking = df["weights"].cumsum()
    ranking_pct = ranking / ranking.max()    
    y_bool = (ranking_pct < percentage)    
    y_bool = y_bool.sort_index()
    
    return y_bool

def compute_metrics(y, y_preds, weights=None, percentage=.667):
    r_test = r2_score(y, y_preds, sample_weight=weights, multioutput='uniform_average')
    print(f"R2 test: {r_test}")

    y_bool = (y.values < np.quantile(y_preds, percentage))
    y_preds_bool = (y_preds < np.quantile(y_preds, percentage))

    acc_test = accuracy_score(y_bool, y_preds_bool, sample_weight=weights)
    print(f"Accuracy Test: {acc_test}")
    
    f2_test = fbeta_score(y_bool, y_preds_bool, beta=2, sample_weight=weights)
    print(f"F2 Test: {f2_test}")
    
    return r_test, acc_test, f2_test

def compute_metrics_urru(y_ur, y_preds_ur, y_ru, y_preds_ru, weights_ur=None, weights_ru=None, percentage_ru=.665468, percentage_ur=.6671753):

    # Create boolean arrays for each dataset
    y_bool_ru = asigna_beneficios(y_ru, weights_ru, percentage_ru)
    y_preds_bool_ru = asigna_beneficios(y_preds_ru, weights_ru, percentage_ru)
    y_bool_ur = asigna_beneficios(y_ur, weights_ur, percentage_ur)
    y_preds_bool_ur = asigna_beneficios(y_preds_ur, weights_ur, percentage_ur)

    # Create single arrays for all data
    y_bool = pd.concat([pd.Series(y_bool_ru), pd.Series(y_bool_ur)])
    y_preds_bool = pd.concat([pd.Series(y_preds_bool_ru), pd.Series(y_preds_bool_ur)])
    
    y = pd.concat([pd.Series(y_ru), pd.Series(y_ur)])
    y_preds = pd.concat([pd.Series(y_preds_ru), pd.Series(y_preds_ur)])
    weights = pd.concat([pd.Series(weights_ru), pd.Series(weights_ur)])

    r_test = r2_score(y, y_preds, sample_weight=weights, multioutput='uniform_average')
    print(f"R2 test: {r_test}")

    acc_test = accuracy_score(y_bool, y_preds_bool, sample_weight=weights)
    print(f"Accuracy Test: {acc_test}")
    
    f2_test = fbeta_score(y_bool, y_preds_bool, beta=2, sample_weight=weights)
    print(f"F2 Test: {f2_test}")
    
    return r_test, acc_test, f2_test


In [11]:
params_grid = {
    'learning_rate': stats.uniform(0.005, 0.1),
    'reg_alpha': stats.uniform(0, 0.1),
    'reg_lambda': stats.uniform(0.7, 0.3),
    'gamma':stats.uniform(0.1, 0.5),
    'max_depth': stats.randint(5, 30),
    'min_child_weight': stats.randint(5, 30),
    'n_estimators': stats.randint(50, 500),
    'colsample_bytree':stats.uniform(0.5, 0.5),
    'subsample': stats.uniform(0.5, .25),
}

casos = {
    "vars_all": {
        "df":df, 
        "vars": vars_conjunto1,
    },
    "vars_min": {
        "df":df, 
        "vars": vars_conjunto2,
    },

}

resultados = {}
for name, params in casos.items():
    print(f"Running model: {name}")
    df_model, vars = params.values()
    # # Nacional
    # results, y_test_preds, y_train_preds, y_test, y_train = fit_xgboost_reg(df_model, "logingreso", vars, params_grid)
    # r2_test, acc_test = compute_metrics(y_test, y_test_preds)
    # resultados[name + "_reg_nac"] = {}
    # resultados[name + "_reg_nac"]["df"] = df_model
    # resultados[name + "_reg_nac"]["results"] = results
    # resultados[name + "_reg_nac"]["best_params"] = results.best_params_
    # resultados[name + "_reg_nac"]["r_test"]  = r2_test
    # resultados[name + "_reg_nac"]["y_test_preds"] = y_test_preds

    # Modelo lineal (para garantizar que esté todo ok)
    model_ru, y_test_preds_ru, y_train_preds_ru, y_test_ru, y_train_ru, w_test_ru, w_train_ru = fit_linear_model(df_model[df_model["UR"]=="Rural"], "logingreso", vars)
    model_ur, y_test_preds_ur, y_train_preds_ur, y_test_ur, y_train_ur, w_test_ur, w_train_ur = fit_linear_model(df_model[df_model["UR"]=="Urbana"], "logingreso", vars)
    print("Linear Model")
    r2_test, acc_test, f2_test = compute_metrics_urru(y_test_ur, y_test_preds_ur, y_test_ru, y_test_preds_ru, weights_ur=w_test_ur, weights_ru=w_test_ru)
    resultados[name + "_linreg_urru"] = {}
    resultados[name + "_linreg_urru"]["df"] = df_model
    resultados[name + "_linreg_urru"]["model"] = (model_ru, model_ur)
    resultados[name + "_linreg_urru"]["best_params"] = {"Rural": (model_ru.intercept_, model_ru.coef_), "Urbana": (model_ur.intercept_, model_ur.coef_)}
    resultados[name + "_linreg_urru"]["r_test"]  = r2_test
    resultados[name + "_linreg_urru"]["f2_test"] = f2_test
    
    # Urbano y Rural estimados por separado
    model_ru, y_test_preds_ru, y_train_preds_ru, y_test_ru, y_train_ru, w_test_ru, w_train_ru = fit_xgboost_reg(df_model[df_model["UR"]=="Rural"], "logingreso", vars, params_grid)
    model_ur, y_test_preds_ur, y_train_preds_ur, y_test_ur, y_train_ur, w_test_ur, w_train_ur = fit_xgboost_reg(df_model[df_model["UR"]=="Urbana"], "logingreso", vars, params_grid)
    print("XGBoost Model")
    r2_test, acc_test, f2_test = compute_metrics_urru(y_test_ur, y_test_preds_ur, y_test_ru, y_test_preds_ru, weights_ur=w_test_ur, weights_ru=w_test_ru)
    resultados[name + "_xgboost_urru"] = {}
    resultados[name + "_xgboost_urru"]["df"] = df_model
    resultados[name + "_xgboost_urru"]["model"] = (model_ru, model_ur)
    resultados[name + "_xgboost_urru"]["best_params"] = {"Rural": model_ru.best_params_, "Urban": model_ur.best_params_}
    resultados[name + "_xgboost_urru"]["r_test"]  = r2_test
    resultados[name + "_xgboost_urru"]["f2_test"] = f2_test

    # results, y_test_preds, r_test, r_train, cm = fit_xgboost_cla(df_model, "pobreza", vars, params_grid)
    # resultados[name + "_cla"] = {}
    # resultados[name + "_cla"]["df"] = df_model
    # resultados[name + "_cla"]["results"] = results
    # resultados[name + "_cla"]["best_params"] = results.best_params_
    # resultados[name + "_cla"]["r_test"]  = r_test
    # resultados[name + "_cla"]["r_train"] = r_train
    # resultados[name + "_cla"]["confusion_matrix"] = cm
    # resultados[name + "_cla"]["y_test_preds"] = y_test_preds


Running model: vars_all
Linear Model
R2 test: 0.423916220664978
Accuracy Test: 0.7848847679259812
F2 Test: 0.8385454430232406
XGBoost Model
R2 test: 0.4154311418533325
Accuracy Test: 0.7740522758856013
F2 Test: 0.8303840799390814
Running model: vars_min
Linear Model
R2 test: 0.3940904140472412
Accuracy Test: 0.7655498977479872
F2 Test: 0.8239218090847
XGBoost Model
R2 test: 0.38318902254104614
Accuracy Test: 0.7627021727585709
F2 Test: 0.8214118845292081
